In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=fe28e8ddc6f53a174089f2ca3fc768600e40ef3560bd40d96da42ff0e8490f57
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from langdetect import detect_langs
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import re
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
from nltk.corpus import stopwords

In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: None]


Importing Dataset

In [ ]:
df=pd.read_csv("/content/data.csv")
df.head()

Exploring Dataset

In [ ]:
df.shape

(5842, 2)

In [ ]:
df.isnull().sum()

Sentence     0
Sentiment    0
dtype: int64

Checking For Unique values

In [ ]:
df["Sentiment"].value_counts()

Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64

**Detecting Language**
Ensuring Only English sentences will be selected

In [ ]:
df["language"]=df["Sentence"].apply(detect_langs)
df.language.head()

0    [en:0.9999959867940897]
1    [en:0.9999987172808331]
2    [en:0.9999956989669443]
3     [en:0.999998204391309]
4     [en:0.999996779626642]
Name: language, dtype: object

In [ ]:
df["lang"]=df["language"].apply(lambda x: str(x).split(":")[0][1:])

In [ ]:
df.head()

,Sentence,Sentiment,language,lang
0,The GeoSolutions technology will leverage Bene...,positive,[en:0.9999964415016573],en
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,[en:0.9999957207935003],en
2,"For the last quarter of 2010 , Componenta 's n...",positive,[en:0.999996488737252],en
3,According to the Finnish-Russian Chamber of Co...,neutral,[en:0.9999973563780854],en
4,The Swedish buyout firm has sold its remaining...,neutral,[en:0.9999983633805146],en


**Pre-processing Text**

In [ ]:
def remove_special_characters(text):
    pattern=r'[^a-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text


def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_text = ' '.join([token for token in tokens if token not in english_stops])
    return filtered_text

english_stops = stopwords.words('english')

def text_preprocessing(text):
    text = text.lower()

    text = remove_special_characters(text)
    text = remove_stopwords(text)
    text = text.strip()
    return text

df["preprocess_text"]=df["Sentence"].apply(text_preprocessing)

In [ ]:
df.head(10)

,Sentence,Sentiment,language,lang,preprocess_text
0,"The GeoSolutions technology will leverage Benefon 's GPS solutions by providing Location Based Search Technology , a Communities Platform , location relevant multimedia content and a new and powerful commercial model .",positive,[en:0.9999959867940897],en,"geosolutions technology leverage benefon 's gps solutions providing location based search technology , communities platform , location relevant multimedia content new powerful commercial model ."
1,"$ESI on lows, down $1.50 to $2.50 BK a real possibility",negative,[en:0.9999987172808331],en,"$ esi lows , $ 1.50 $ 2.50 bk real possibility"
2,"For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .",positive,[en:0.9999956989669443],en,"last quarter 2010 , componenta 's net sales doubled eur131m eur76m period year earlier , moved zero pre-tax profit pre-tax loss eur7m ."
3,"According to the Finnish-Russian Chamber of Commerce , all the major construction companies of Finland are operating in Russia .",neutral,[en:0.999998204391309],en,"according finnish-russian chamber commerce , major construction companies finland operating russia ."
4,"The Swedish buyout firm has sold its remaining 22.4 percent stake , almost eighteen months after taking the company public in Finland .",neutral,[en:0.999996779626642],en,"swedish buyout firm sold remaining 22.4 percent stake , almost eighteen months taking company public finland ."
5,$SPY wouldn't be surprised to see a green close,positive,[en:0.9999949180063873],en,$ spy would n't surprised see green close
6,Shell's $70 Billion BG Deal Meets Shareholder Skepticism,negative,[en:0.9999954974177123],en,shell 's $ 70 billion bg deal meets shareholder skepticism
7,"SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANGE RELEASE OCTOBER 14 , 2008 AT 2:45 PM The Company updates its full year outlook and estimates its results to remain at loss for the full year .",negative,[en:0.9999974031175379],en,"ssh communications security corp stock exchange release october 14 , 2008 2:45 pm company updates full year outlook estimates results remain loss full year ."
8,Kone 's net sales rose by some 14 % year-on-year in the first nine months of 2008 .,positive,[en:0.9999964572331481],en,kone 's net sales rose 14 % year-on-year first nine months 2008 .
9,"The Stockmann department store will have a total floor space of over 8,000 square metres and Stockmann 's investment in the project will have a price tag of about EUR 12 million .",neutral,[en:0.9999964607303059],en,"stockmann department store total floor space 8,000 square metres stockmann 's investment project price tag eur 12 million ."


**Splitting Data into Features for Training and Testing**

In [ ]:
X = df['preprocess_text']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Linear SVM Model**

In [ ]:
lin_svc_pipe = Pipeline(([('tfidf',TfidfVectorizer()),('clf',LinearSVC(dual=True))]))
lin_svc_pipe.fit(X_train,y_train)
lin_svc_pred = lin_svc_pipe.predict(X_test)
print(confusion_matrix(y_test,lin_svc_pred))
print()
print(classification_report(y_test,lin_svc_pred))

[[ 68 147  54]
 [117 722  86]
 [ 21 157 381]]

              precision    recall  f1-score   support

    negative       0.33      0.25      0.29       269
     neutral       0.70      0.78      0.74       925
    positive       0.73      0.68      0.71       559

    accuracy                           0.67      1753
   macro avg       0.59      0.57      0.58      1753
weighted avg       0.66      0.67      0.66      1753



**Logistc Regression Model**

In [ ]:
lr_pipe = Pipeline(([('tfidf',TfidfVectorizer()),('clf',LogisticRegression())]))
lr_pipe.fit(X_train,y_train)
lr_pipe_pred = lr_pipe.predict(X_test)
print(confusion_matrix(y_test,lr_pipe_pred))
print()
print(classification_report(y_test,lr_pipe_pred))

[[ 38 174  57]
 [ 36 834  55]
 [  8 221 330]]

              precision    recall  f1-score   support

    negative       0.46      0.14      0.22       269
     neutral       0.68      0.90      0.77       925
    positive       0.75      0.59      0.66       559

    accuracy                           0.69      1753
   macro avg       0.63      0.54      0.55      1753
weighted avg       0.67      0.69      0.65      1753



**Naive-Bayes Model**

In [ ]:
nb_pipe = Pipeline(([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())]))
nb_pipe.fit(X_train,y_train)
nb_pipe_pred = nb_pipe.predict(X_test)
print(confusion_matrix(y_test,nb_pipe_pred))
print()
print(classification_report(y_test,nb_pipe_pred))

[[ 10 192  67]
 [  1 901  23]
 [  1 330 228]]

              precision    recall  f1-score   support

    negative       0.83      0.04      0.07       269
     neutral       0.63      0.97      0.77       925
    positive       0.72      0.41      0.52       559

    accuracy                           0.65      1753
   macro avg       0.73      0.47      0.45      1753
weighted avg       0.69      0.65      0.58      1753



**Evaluating Model performance**

In [90]:
print('Linear SVC : ',accuracy_score(y_test,lin_svc_pred))
print('Logistic Regression : ',accuracy_score(y_test,lr_pipe_pred))
print('Naive Bayes : ',accuracy_score(y_test,nb_pipe_pred))


Linear SVC :  0.667997718197376
Logistic Regression :  0.6856816885339418
Naive Bayes :  0.6497432972047917


**Testing for new data**

In [87]:
new_x=['the ceo of  a company is interested in issuing 15000 stocks next month','the company is doing very well and expanding in few months']
nx=[text_preprocessing(text) for text in new_x]
lin_svc_pipe.predict(nx)



array(['neutral', 'positive'], dtype=object)

In [88]:
lr_pipe.predict(nx)

array(['neutral', 'positive'], dtype=object)

In [89]:
nb_pipe.predict(nx)

array(['neutral', 'neutral'], dtype='<U8')